In [ ]:
pip install python-dotenv

In [ ]:
# import os
# from dotenv import load_dotenv, find_dotenv
# _= load_dotenv(find_dotenv())

# ap =os.getenv("MY_VAR")

In [ ]:
# !pip install langchain
# !pip install openai

In [ ]:
!pip install -U langchain-openai
!pip install langchain

In [ ]:
# prompt: 어떻게 api_key를 넣는가?
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

# Load the environment variables from the .env file
load_dotenv(find_dotenv())

# Get the OpenAI API key from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
api_key = 'YOUR_API_KEY'

In [ ]:
url = "https://github.com/tlorans/ClimateRisks/blob/main/DEXUSEU.csv?raw=true"

import pandas as pd
data = pd.read_csv(url)
data.rename(columns = {"DEXUSEU":"Exchange Rate Dollar to Euro"}, inplace = True)
data.to_csv("exchange_rate.csv", index = None)

In [ ]:
!pip install langchain-community # Install the langchain_community package

In [ ]:
from langchain.document_loaders import CSVLoader

file = "exchange_rate.csv"
loader = CSVLoader(file_path = file)
data = loader.load()
print(data[0])

Custom Tools

In [ ]:
!pip uninstall -y docarray
!pip install docarray
!pip install tiktoken

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings # import embeddings

# Initialize embeddings object
embeddings = OpenAIEmbeddings(
    openai_api_key = api_key
)

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings # add embeddings argument
).from_loaders([loader])

In [ ]:
from langchain.llms import OpenAI # import the OpenAI class

llm = OpenAI(
    temperature=0,
    openai_api_key = api_key
)

In [ ]:
index.query(
    "What is the last exchange rate? Gives the date",
    llm = llm
    )

In [ ]:
# TODO: docarray 버전 충돌 문제로 인해 실행이 안됨

In [ ]:
# pip install -U docarray==0.21.0

In [ ]:
from docarray import DocumentArray, Document

# query_text를 사용하여 Document 객체 생성
query_text = "What is the last exchange rate? Gives the date"
query_doc = Document(text=query_text)

# Document 객체를 DocumentArray에 추가
docs = DocumentArray([query_doc])

# DocumentArray를 사용하여 쿼리 실행
response = index.query(docs)

print(response)



In [ ]:
from langchain.tools import Tool

knowledge_tool = Tool(
    name = "FX Rate",
    func = index.query,
    description = "Useful for when you need to search for the exchange rate. Provides your input as a search query."
)

In [ ]:
from langchain.agents import AgentType
from langchain.agents import load_tools, initialize_agent
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature = 0.)


tools = [knowledge_tool]

agent = initialize_agent(
    tools,
    chat,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

agent.run("What was the exchange rate in April 2022?")

Interacting with Other tools

In [ ]:
!pip install duckduckgo-search

In [ ]:
tools = load_tools(["llm-math"], llm = chat)

from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

duckduckgo_tool = Tool(
    name = 'DuckDuckGo Search',
    func = search.run,
    description = "Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

tools.append(knowledge_tool)
tools.append(duckduckgo_tool)

agent = initialize_agent(
    tools,
    chat,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

In [ ]:
query = """Process as follow:\n\
1. Search for Tesla's revenue in 2022. \n\
2. Find the exchange rate in December 2022. \n\
3. Multiply Tesla's revenue in 2022 with the exchange rate. \n\
"""
agent.run(query)

연습

In [ ]:
url = "	https://pasteur.epa.gov/uploads/10.23719/1528686/SupplyChainGHGEmissionFactors_v1.2_NAICS_CO2e_USD2021.csv"

import pandas as pd
emissions = pd.read_csv(url)

emissions.to_csv("emissions_factors.csv", index = None)

file = "emissions_factors.csv"
loader = CSVLoader(file_path = file)
data = loader.load()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
index.query("What is the emission factor with margins for automobile industry?")

In [ ]:
!pip install openai
!pip install langchain
!pip install docarray
!pip install tiktoken
!pip install duckduckgo-search

In [ ]:
from langchain.tools import Tool

knowledge_tool = Tool(
    name = "Supply Chain Emissions Factors",
    func = index.query,
    description = "Useful for when you need to find the emissions supply-chain for a specific industry. You want to return the emissions factors from the \
    most related industry."
)

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

duckduckgo_tool = Tool(
    name = 'DuckDuckGo Search',
    func = search.run,
    description = "Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

In [ ]:
from langchain.agents import AgentType
from langchain.agents import load_tools, initialize_agent
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature = 0.)
tools = load_tools(["llm-math"], llm = chat)

tools.append(knowledge_tool)
tools.append(duckduckgo_tool)

agent = initialize_agent(
    tools,
    chat,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

In [ ]:
agent.run("""Please follow the following process: \n\
1. Find Tesla's main activity and last available revenue.\n\
2. Find the emissions factor with margin corresponding to Tesla's activity\n\
3. Multiply the emissions factor with the revenue\n\
Result:
""")